In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df_bhavcopy = pd.read_csv('df_bhavcopy.csv').set_index('Unnamed: 0')
df_bhavcopy.head(4)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,ADJ_CLOSE
Unnamed: 0,,,,,,,,,,,,,,
0,20MICRONS,EQ,62.05,63.90,61.4,62.50,63.50,62.25,70533,4393274.15,02-JAN-2012,621.0,INE144J01019,NaN
1,3IINFOTECH,EQ,11.85,12.25,11.7,12.05,12.15,11.65,1528247,18131238.80,02-JAN-2012,5193.0,INE748C01020,NaN
2,3MINDIA,EQ,3588.80,3588.80,3450.1,3450.30,3450.10,3544.05,44,153583.70,02-JAN-2012,32.0,INE470A01017,NaN
3,A2ZMES,EQ,87.95,93.80,87.1,89.15,89.50,88.00,124736,11189791.15,02-JAN-2012,3493.0,INE619I01012,NaN


In [4]:
# df_bhavcopy['year'] = pd.DatetimeIndex(df_bhavcopy['TIMESTAMP']).year
# df_bhavcopy['month'] = pd.DatetimeIndex(df_bhavcopy['TIMESTAMP']).month
df_bhavcopy['Yr_Month'] = pd.DatetimeIndex(df_bhavcopy['TIMESTAMP']).strftime('%Y_%m')
df_bhavcopy['DATE'] = pd.to_datetime(df_bhavcopy['TIMESTAMP'])

In [5]:
df_bhavcopy.head(2)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,ADJ_CLOSE,Yr_Month,DATE
Unnamed: 0,,,,,,,,,,,,,,,,
0,20MICRONS,EQ,62.05,63.90,61.4,62.50,63.50,62.25,70533,4393274.15,02-JAN-2012,621.0,INE144J01019,NaN,2012_01,2012-01-02
1,3IINFOTECH,EQ,11.85,12.25,11.7,12.05,12.15,11.65,1528247,18131238.80,02-JAN-2012,5193.0,INE748C01020,NaN,2012_01,2012-01-02


In [6]:
filepath_ca = 'C:\\Saiyad ADT\\Learnnig\\Sensex\\Screener1\\Data_2021_1121\\Corporate_Actions_BSE_2017_2020.csv'    
df_split = pd.read_csv(filepath_ca, header=0)
df_split.head(3)

,Security Code,Security Name,Company Name,Ex Date,Purpose,Record Date,BC Start Date,BC End Date\t,ND Start Date,ND End Date,Actual Payment Date
0,500228,JSWSTEEL,JSW STEEL LTD.,04-Jan-17,Stock Split From Rs.10/- to Rs.1/-,05-Jan-17,-,-,29-Dec-16,04-Jan-17,-
1,530759,STERTOOLS,STERLING TOOLS LTD.-$,09-Jan-17,Stock Split From Rs.10/- to Rs.2/-,10-Jan-17,-,-,03-Jan-17,09-Jan-17,-
2,511736,USHDI,USHDEV INTERNATIONAL LTD.,30-Jan-17,Stock Split From Rs.10/- to Rs.1/-,31-Jan-17,-,-,23-Jan-17,30-Jan-17,-


In [7]:
# Adjusted Stock price for Split and Bonus
def Adjust_stock_splits(df_filter):
    filepath_ca = 'C:\\Saiyad ADT\\Learnnig\\Sensex\\Screener1\\Data_2021_1121\\Corporate_Actions_BSE_2012_2021.csv'    
    df_split = pd.read_csv(filepath_ca, header=0)
    df_split = df_split.rename(columns = {"Security Name":"Tikr","Ex Date":"Ex_Date"})
    df_split['Category'] = ''
    df_split['Category'][df_split['Purpose'].str.contains('Split')] = 'Split'
    df_split['Category'][df_split['Purpose'].str.contains('Bonus')] = 'Bonus'
    
    df_split['Purpose'] = df_split['Purpose'].replace(" to ",":" ,regex = True)
    df_split['Purpose'] = df_split['Purpose'].str.replace('[a-z]+','')
    df_split['Purpose'] = df_split['Purpose'].str.replace('[A-Z]+','')
    charsToRemove = ["$", ".", "€","/","-"]
    df_split['Purpose'] = df_split['Purpose'].str.replace(r"[{}]+".format("".join([re.escape(x) for x in charsToRemove])),"", regex=True)
    df_split['Purpose'] = df_split['Purpose'].str.strip()
    df_split['Ex_Date'] = pd.to_datetime(df_split['Ex_Date'], format = '%d-%b-%y')
    df_split[['Pre', 'Post']] = df_split['Purpose'].str.split(':', 1, expand=True)
    #df_split['Multiplier'] = pd.to_numeric(df_split['Pre'],errors='coerce')/(pd.to_numeric(df_split['Pre'],errors='coerce') + pd.to_numeric(df_split['Post'],errors='coerce'))
    
    #df_split.loc[df_split['Category'] == , 'difficulty'] = 'Easy'

    df_split['Multiplier'] = np.where(df_split['Category'] == 'Split', 
                                        pd.to_numeric(df_split['Post'],errors='coerce')/pd.to_numeric(df_split['Pre'],errors='coerce'), 
                                        pd.to_numeric(df_split['Pre'],errors='coerce')/(pd.to_numeric(df_split['Pre'],errors='coerce') + pd.to_numeric(df_split['Post'],errors='coerce')))
                        

    #df_split = df_split[df_split['Tikr'] == tikr]
    df_split_adj = pd.merge(df_filter, df_split[['Tikr','Ex_Date','Multiplier']], how='left',left_on=['SYMBOL'], right_on=['Tikr'])
    # Pending - To map the numbers and divide the numbers before Start Date
    #sample = df_split_adj.head(2000)
    for i in (['OPEN','HIGH','LOW','CLOSE']):
        df_split_adj[i] = np.where(df_split_adj['DATE'] < df_split_adj['Ex_Date'],df_split_adj[i]*df_split_adj['Multiplier'],df_split_adj[i])
    return df_split_adj

df_bhav_split_adjusted = Adjust_stock_splits(df_bhavcopy)
df_bhav_split_adjusted.to_csv('df_bhavcopy_split_adjusted.csv')


C:\Users\hashmy\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\hashmy\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
df_bhav_split_adjusted[df_bhav_split_adjusted['SYMBOL'] == 'WIPRO']

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,ADJ_CLOSE,Yr_Month,DATE,Tikr,Ex_Date,Multiplier
1830,WIPRO,EQ,199.50,200.950,197.70,199.500,398.45,398.70,640672,2.556089e+08,02-JAN-2012,16876.0,INE075A01022,NaN,2012_01,2012-01-02,WIPRO,2017-06-13,0.50
1831,WIPRO,EQ,99.75,100.475,98.85,99.750,398.45,398.70,640672,2.556089e+08,02-JAN-2012,16876.0,INE075A01022,NaN,2012_01,2012-01-02,WIPRO,2019-03-06,0.25
1832,WIPRO,EQ,199.50,200.950,197.70,199.500,398.45,398.70,640672,2.556089e+08,02-JAN-2012,16876.0,INE075A01022,NaN,2012_01,2012-01-02,WIPRO,2017-06-13,0.50
1833,WIPRO,EQ,99.75,100.475,98.85,99.750,398.45,398.70,640672,2.556089e+08,02-JAN-2012,16876.0,INE075A01022,NaN,2012_01,2012-01-02,WIPRO,2019-03-06,0.25
3616,WIPRO,EQ,206.50,207.300,205.00,205.925,412.90,413.90,1059014,4.367454e+08,01-FEB-2012,19823.0,INE075A01022,NaN,2012_02,2012-02-01,WIPRO,2017-06-13,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229697,WIPRO,EQ,651.10,656.950,647.50,655.450,654.30,646.75,4401233,2.870603e+09,01-NOV-2021,96834.0,INE075A01022,NaN,2021_11,2021-11-01,WIPRO,2019-03-06,0.25
231912,WIPRO,EQ,642.60,645.950,633.25,634.800,635.60,637.25,4726822,3.018105e+09,01-DEC-2021,117863.0,INE075A01022,NaN,2021_12,2021-12-01,WIPRO,2017-06-13,0.50
231913,WIPRO,EQ,642.60,645.950,633.25,634.800,635.60,637.25,4726822,3.018105e+09,01-DEC-2021,117863.0,INE075A01022,NaN,2021_12,2021-12-01,WIPRO,2019-03-06,0.25
231914,WIPRO,EQ,642.60,645.950,633.25,634.800,635.60,637.25,4726822,3.018105e+09,01-DEC-2021,117863.0,INE075A01022,NaN,2021_12,2021-12-01,WIPRO,2017-06-13,0.50


In [9]:
#Getting average of the 4 values
df_bhav_split_adjusted['AVG'] = df_bhav_split_adjusted[['OPEN','HIGH','LOW','CLOSE']].mean(axis=1)

df_bhav_pivot = pd.pivot_table(df_bhav_split_adjusted, index=['SYMBOL', 'SERIES'], columns=['Yr_Month'], values=['AVG'], aggfunc='sum')
# Replacing spaces from the column names and joining the two layer of column names into one.
df_bhav_pivot.columns =[str(s2) + str('_') + s1  for (s1,s2) in df_bhav_pivot.columns.tolist()]
df_bhav_pivot.reset_index(inplace=True)
# Reordering columns by name so that all similar column get together
df_bhav_pivot = df_bhav_pivot.reindex(sorted(df_bhav_pivot.columns), axis=1)
df_bhav_pivot.head(3)

,2012_01_AVG,2012_02_AVG,2012_03_AVG,2012_04_AVG,2012_05_AVG,2012_06_AVG,2012_07_AVG,2012_08_AVG,2012_09_AVG,2012_10_AVG,...,2021_05_AVG,2021_06_AVG,2021_07_AVG,2021_08_AVG,2021_09_AVG,2021_10_AVG,2021_11_AVG,2021_12_AVG,SERIES,SYMBOL
0,62.4625,63.9875,79.1125,82.725,89.375,89.9875,86.575,96.9875,117.7625,115.6625,...,77.75,125.0000,119.9250,133.95,108.225,122.325,117.325,117.375,EQ,20MICRONS
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.10,15.3375,23.0375,23.00,34.200,49.200,55.400,55.400,EQ,21STCENMGM
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.775,EQ,3IINFOLTD


In [10]:
df_bhav_yearly = pd.DataFrame()
for i in range(2012,2022):
    #print(i)
    df_bhav_yearly[i] = df_bhav_pivot.filter(regex=str(i)).mean(axis=1)
df_bhav_yearly['Tikr'] = df_bhav_pivot['SYMBOL']
df_bhav_yearly.head()

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Tikr
0,95.362500,76.942708,64.275000,63.268750,64.902083,82.964583,92.670833,75.133333,60.702083,101.847917,20MICRONS
1,NaN,NaN,NaN,NaN,NaN,NaN,33.429688,16.686364,10.925000,26.258333,21STCENMGM
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.775000,3IINFOLTD
3,13.046875,5.323611,8.043750,5.565909,5.004167,4.688542,4.785417,2.892708,2.442708,7.418750,3IINFOTECH
4,3822.328125,3523.364583,4749.711458,9002.241667,12208.872917,13077.967708,21055.735417,21991.759375,19984.603125,24529.438542,3MINDIA


In [11]:
df_bhav_yearly[df_bhav_yearly['Tikr'] == 'NIFTY50']

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Tikr
1577,5361.678422,5885.784157,7378.285267,8340.659102,8156.784069,9502.199972,10825.780695,11480.707986,10907.246538,15777.836304,NIFTY50


In [12]:
len(df_bhav_yearly.columns) - 2
cols = df_bhav_yearly.columns
# Percentae calclation
def percentage_change(col1,col2):
    try:
        out = ((col2 - col1) / col1) * 100
    except:
        out = np.nan
    return out
    #return ((df_sales.loc[:,col2] - df_sales.loc[:,col1])/df_sales.loc[:,col1]) * 100

df_bhav_delta = pd.DataFrame()

# Looping over columns to get the percentage change YoY
for i in range(len(df_bhav_yearly.columns) - 1):
    #print(cols[i])
    if i > 2:
        df_bhav_delta[cols[i]] = percentage_change(df_bhav_yearly[cols[i-3]],df_bhav_yearly[cols[i]])
df_bhav_delta['Tikr'] = df_bhav_yearly['Tikr']
df_bhav_delta.head(4)


,2015,2016,2017,2018,2019,2020,2021,Tikr
0,-33.654476,-15.648819,29.077531,46.471731,15.764132,-26.833739,9.902882,20MICRONS
1,NaN,NaN,NaN,NaN,NaN,NaN,-21.452053,21STCENMGM
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3IINFOLTD
3,-57.339140,-6.000522,-41.711992,-14.022730,-42.194005,-47.900467,55.028298,3IINFOTECH


In [13]:
# # Averaging the years month over month changes to get an average return

# df_bhav_delta_monthly = pd.DataFrame()
# for i in range(2015,2022):
#     #print(i)
#     df_bhav_delta_monthly[i] = df_bhav_delta.filter(regex=str(i)).mean(axis=1)

# #df_bhav_delta[''] = df.filter(regex='spike')
# df_bhav_delta_monthly['Tikr'] = df_bhav_delta['Tikr']
# df_bhav_delta_monthly.head(4)
df_bhav_delta.to_csv('df_bhav_delta_monthly.csv')

# Initially we were calculating the MoM returns and averaging them. but that did not give any result in modeling..
# So now we moved to 3 year returns, expecting, this will reduce the noise in the system

In [14]:
# Normalization by subtracting the index growth
#df_bhav_delta[df_bhav_delta['Tikr'] == 'WIPRO']
df_bhav_delta[df_bhav_delta['Tikr'] == 'NIFTY50']
# These numbers are pretty high, was the index really performing this well YoY ?

,2015,2016,2017,2018,2019,2020,2021,Tikr
1577,55.5606,38.584492,28.78602,29.795266,40.750422,14.78654,45.743173,NIFTY50


In [15]:
df_bhav_yearly[df_bhav_yearly['Tikr'] == 'NIFTY50']

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Tikr
1577,5361.678422,5885.784157,7378.285267,8340.659102,8156.784069,9502.199972,10825.780695,11480.707986,10907.246538,15777.836304,NIFTY50


In [19]:
df_bhav_delta[df_bhav_delta['Tikr'] == 'RAIN']    

,2015,2016,2017,2018,2019,2020,2021,Tikr
1828,NaN,10.171558,236.491842,638.763083,162.482544,-35.989132,-32.814939,RAIN
